In [2]:
from bs4 import BeautifulSoup
import urllib
import csv
import re
import numpy as np
import pandas as pd

In [3]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
option = webdriver.ChromeOptions()
option.add_argument("--incognito")
driver = webdriver.Chrome(executable_path='chromedriver.exe', options=option)

In [4]:
import time
driver.get("https://www.sulekha.com/maids/bangalore")
#n is the number of total result available /10
n = 41
for x in range(n):
    more_buttons = driver.find_elements_by_class_name("loadlist")
    if more_buttons[0].is_displayed():
        driver.execute_script("arguments[0].click();", more_buttons[0])
        time.sleep(1)
page_source = driver.page_source

In [9]:
def innerHTML(element):
    return element.decode_contents(formatter="html")

def get_name(body):
    c = body.find('div', {'class':'head'})
    if c:
        return c.a.h3.string
    else :
        return "NA"

def get_phone_number(body):
    c = body.find('b', {'class':'icon-phone f-icon isbvn'})
    if c:
        return c.string
    else :
        return "NA"

def get_score(body):
    c = body.find('span', {'class':'score'})
    if c:
        return c.strong.text
    else :
        return "NA"

def get_stars(body):
    
    if body.find('span', {'class':'star5'}):
        return 5
    elif body.find('span', {'class':'star4-5'}):
        return 4.5
    elif body.find('span', {'class':'star4'}):
        return 4
    elif body.find('span', {'class':'star3-5'}):
        return 3.5    
    elif body.find('span', {'class':'star3'}):
        return 3    
    elif body.find('span', {'class':'star2-5'}):
        return 2.5    
    elif body.find('span', {'class':'star2'}):
        return 2    
    elif body.find('span', {'class':'star1-5'}):
        return 1.5    
    elif body.find('span', {'class':'star1'}):
        return 1    
    elif body.find('span', {'class':'star0-5'}):
        return 0.5   
    elif body.find('span', {'class':'star0'}):
        return 0
    else :
        return "NA"

def get_reviews(body):
    c = body.find('div',{'class':'ratings'})
    if c:
        k = c.find_all('span')
        if len(k)>2:
            string  = k[2].text.strip()
            if string.find('Review')!= -1 or string.find('Reviews')!=-1:
                return string
            else :
                return "NA"
        else :
            return "NA"
def get_tag(body):
    c =body.find('p', {'class':'icon-tag f-icon'}) 
    if c:
        return c.string
    else :
        return "NA"

def get_address(body):
    c =body.find('span', {'class':'location'})
    if c:
        return c.text.strip()
    else :
        return "NA"

def get_decription(body):
    c =body.find('div', {'class':'body'})
    if c:
        return c.text.strip()
    else :
        return "NA"    

In [10]:
def get_href(body):
    c = body.find('td',{'class':'wraper'})
    if c:
        return c.a.get('href')
    else :
        return "NA"


In [11]:
soup = BeautifulSoup(page_source, "lxml")
print(bool(soup.find('div', {'class':'head'}).a.h3.string))
services = soup.find_all('li', {'class': 'list-item'})
scraped_data = []
for service_html in services:
    dict_service = {}
    name = get_name(service_html)
    phone = get_phone_number(service_html)
    score = get_score(service_html)
    stars = get_stars(service_html)
    reviews = get_reviews(service_html)
    tag = get_tag(service_html)
    address = get_address(service_html)
    description =get_decription(service_html)
    url = get_href(service_html)
    dict_service['Name'] = name
    dict_service['Phone'] = phone
    dict_service['Score'] = score
    dict_service['Tag'] = tag
    dict_service['Address'] = address
    dict_service['Description'] = description
    dict_service['Review_Stars'] = stars
    dict_service['Review_Count'] = reviews
    dict_service['Url'] = url
    scraped_data.append(dict_service)
dataFrame = pd.DataFrame.from_dict(scraped_data)
# dataFrame.to_csv('sulekha_housemaid_bangalore.csv', index=False)

True


In [12]:
dataFrame

,Name,Phone,Score,Tag,Address,Description,Review_Stars,Review_Count,Url
0,Charulatha Agency,+918048093043,6.9,"Maids, Domestic help services","9th Cross, Whitefield, Bangalore",we address your need with atmost care .feel fr...,3.5,3 Reviews,NA
1,Infant Services,+918048047704,8.4,"Maids, Domestic help services","Begur Road, Bommanahalli, Bangalore",infant services are into domestic help service...,3.5,2 Reviews,NA
2,A S R Groups,+919595210347,8.5,"Maids, Domestic help services","Manjunath Nagar, Rajaji Nagar, Bangalore","A.S.R. Groups, established in 2012, provides s...",5.0,5 Reviews,NA
3,Sandria J Manpower Agency,+918048102126,9.0,"Maids, Domestic help services","Thomas Town Post, Kammanahalli, Bangalore",Sandria.j manpower agency domestic help servic...,5.0,1 Review,NA
4,free lancer,+918048102203,7.5,"Maids, Domestic help services","Bangalore, Kammanahalli, Bangalore",Rani manpower agency Demestic service we are h...,3.0,4 Reviews,NA
...,...,...,...,...,...,...,...,...,...
117,Lakshmi Domestic Services,NA,NA,"Maids, Domestic help services","Andhra Colony, Vivek Nagar, Bangalore",We are dealing in this business for past sever...,0.0,NA,NA
118,St. Mary's Business Solutions,NA,NA,"Maids, Domestic help services","Raj Palya, Kammanahalli, Bangalore",We are in this business for several years. We ...,5.0,NA,NA
119,Cook Manoj Kormangala Ejipura,NA,NA,"Maids, Domestic help services","Ejipura Signal, Koramangala","3rd Cross, Ejipura Signal, Koramangala, Bangal...",0.0,NA,NA
120,Charulatha Agency,+918048093043,6.9,"Maids, Domestic help services","9th Cross, Whitefield, Bangalore",we address your need with atmost care .feel fr...,3.5,3 Reviews,NA
